In [6]:
def Amazon_meta(dataset_name, data_maps):
    '''
    asin - ID of the product, e.g. 0000031852
    --"asin": "0000031852",
    title - name of the product
    --"title": "Girls Ballet Tutu Zebra Hot Pink",
    description
    price - price in US dollars (at time of crawl)
    --"price": 3.17,
    imUrl - url of the product image (str)
    --"imUrl": "http://ecx.images-amazon.com/images/I/51fAmVkTbyL._SY300_.jpg",
    related - related products (also bought, also viewed, bought together, buy after viewing)
    --"related":{
        "also_bought": ["B00JHONN1S"],
        "also_viewed": ["B002BZX8Z6"],
        "bought_together": ["B002BZX8Z6"]
    },
    salesRank - sales rank information
    --"salesRank": {"Toys & Games": 211836}
    brand - brand name
    --"brand": "Coxlures",
    categories - list of categories the product belongs to
    --"categories": [["Sports & Outdoors", "Other Sports", "Dance"]]
    '''
    datas = {}
    meta_flie = './raw/meta_' + str(dataset_name) + '.json'
    item_asins = set(data_maps['str2id'].keys()) # 高速なルックアップのためにセットに変換

    for info in parse_meta(meta_flie):
        if info.get('asin') in item_asins:
            datas[info['asin']] = info
    return datas

import json
from tqdm import tqdm

def parse_meta(path): # for Amazon
    inter_list = []
    with open(path, 'r') as f:
        for line in tqdm(f):
            if line and line.strip():
                inter_list.append(json.loads(line))
    return inter_list


# def parse_meta(path): # for Amazon
#     g = gzip.open(path, 'rb')
#     inter_list = []
#     for l in tqdm(g):
#         if l and l.strip():
#             inter_list.append(json.loads(l.decode()) )

#     return inter_list

In [7]:
# 2_fetch_metadata.py

import json
import os
from collections import defaultdict

# --- 設定項目 ---
HANDLED_DIR = "./handled"

    # 0: "Clothing_Shoes_and_Jewelry",
    # 1: "Sports_and_Outdoors",
domains = {
    0: "Clothing_Shoes_and_Jewelry",
    1: "Sports_and_Outdoors",
    2: "AMAZON_FASHION"
}
# --- ここまで ---

# --- ステップ1: id_map.jsonを読み込む ---
id_map_path = os.path.join(HANDLED_DIR, "id_map.json")
if not os.path.exists(id_map_path):
    print(f"Error: '{id_map_path}' not found. Please run '1_process_interactions.py' first.")
    exit()

with open(id_map_path, "r") as f:
    map_dict = json.load(f)
item_dict = map_dict["item_dict"]

# --- ステップ2: 各ドメインのメタデータを取得・保存 ---
print("\n--- Fetching and saving metadata for filtered items ---")
for domain_id, domain_name in domains.items():
    meta_file_path = os.path.join(HANDLED_DIR, f"item2attributes_{chr(ord('A') + domain_id)}.json")
    
    if not os.path.exists(meta_file_path):
        print(f"Fetching metadata for domain '{domain_name}'...")
        if str(domain_id) in item_dict:
            # フィルタリング後のitem IDのみを対象にメタデータを取得
            meta_data = Amazon_meta(domain_name, item_dict[str(domain_id)])
            with open(meta_file_path, 'w') as out:
                json.dump(meta_data, out)
            print(f"Saved metadata to {meta_file_path}")
        else:
            print(f"No items left for domain '{domain_name}' after filtering, skipping metadata.")
    else:
        print(f"--- Found existing metadata file for '{domain_name}', skipping ---")

print("\nMetadata fetching finished successfully!")


--- Fetching and saving metadata for filtered items ---
--- Found existing metadata file for 'Clothing_Shoes_and_Jewelry', skipping ---
--- Found existing metadata file for 'Sports_and_Outdoors', skipping ---
Fetching metadata for domain 'AMAZON_FASHION'...


0it [00:00, ?it/s]

186637it [00:01, 125668.34it/s]


Saved metadata to ./handled/item2attributes_C.json

Metadata fetching finished successfully!
